In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn import Sigmoid
import torch
import os

# MODEL_NAME = "hotchpotch/tmp-mb-jp-30m-reranker"
MODEL_NAME = "../outputs/exp337"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.to(device)
model.eval()

if device == "cuda":
    model.half()


/home/hotchpotch/src/github.com/hotchpotch/yart/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [2]:
query = "感動的な映画について"
passages = [
    "深いテーマを持ちながらも、観る人の心を揺さぶる名作。登場人物の心情描写が秀逸で、ラストは涙なしでは見られない。",
    "重要なメッセージ性は評価できるが、暗い話が続くので気分が落ち込んでしまった。もう少し明るい要素があればよかった。",
    "どうにもリアリティに欠ける展開が気になった。もっと深みのある人間ドラマが見たかった。",
    "アクションシーンが楽しすぎる。見ていて飽きない。ストーリーはシンプルだが、それが逆に良い。",
]
inputs = tokenizer(
    [(query, passage) for passage in passages],
    padding=True,
    truncation=True,
    max_length=8192,
    return_tensors="pt",
)
inputs = {k: v.to(device) for k, v in inputs.items()}
logits = model(**inputs).logits
activation = Sigmoid()
scores = activation(logits).squeeze().tolist()

# sort by score
sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
# show sorted results
for i in sorted_indices:
    print(f"Passage: {passages[i]}")
    print(f"Score: {scores[i]}")
    print()


Passage: 深いテーマを持ちながらも、観る人の心を揺さぶる名作。登場人物の心情描写が秀逸で、ラストは涙なしでは見られない。
Score: 0.1231689453125

Passage: アクションシーンが楽しすぎる。見ていて飽きない。ストーリーはシンプルだが、それが逆に良い。
Score: 0.058349609375

Passage: どうにもリアリティに欠ける展開が気になった。もっと深みのある人間ドラマが見たかった。
Score: 0.04443359375

Passage: 重要なメッセージ性は評価できるが、暗い話が続くので気分が落ち込んでしまった。もう少し明るい要素があればよかった。
Score: 0.0257110595703125

